In [ ]:
// Referenziere notwendige Software Pakete
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FSharp.Stats, 0.4.7"
#r "nuget: FsODE, 0.0.1"

// ... und öffne die darin enthaltenen Module
open FsODE
open Plotly.NET
open Plotly.NET.LayoutObjects
open Plotly.NET.StyleParam
open FSharp.Stats

Installed Packages FSharp.Stats, 0.4.7 FsODE, 0.0.1 Plotly.NET.Interactive, 3.0.2

# Tag 2 - Modellierung von Wachstum

### Wiederholung Wachstumsmodellierung

Die Beschreibung und Analyse von Wachstum gehört seit jeher zu den Standardexperimenten in der Biologie. Oft sind Wachstumsexperimente simpel aufgebaut: Es gibt eine Kontrollprobe und eine Probe die irgendeiner Art von Stress ausgesetzt wird (anderes Wachstumsmedium, andere Umwelteinflüsse, andere Genotypen oder Mutationen). Über die Zeit wird die Zellzahl in beiden Proben gemessen und in ein Diagramm eingetragen. 

Gestern wurde Ihnen eine der einfachsten Differentialgleichungen gezeigt die Wachstum beschreibt: $\frac{dN}{dt}=rN$ mit der integrierten Lösung $N = N_0 *e^{rt}$

Schauen wir uns die Differentialgleichung nochmals detailliert an. Die Änderung der Zellzahl über die Zeit $\frac{dN}{dt}$ ist die Steigung der Kurve die die Zunahme der Zellzahl beschreibt. Diese Steigung ist ausschließlich abhängig von der Zellzahl selbst (_N_) und einem Wachstumsparameter (_r_). Je mehr Zellen vorhanden sind, desto schneller kann die Population wachsen, mehr Annahmen trifft dieses Modell nicht. Gestern haben Sie den Solver kennengelernt, der es ermöglicht eine Differentialgleichung zu formulieren und dadurch den Funktionsverlauf der "echten" integrierten Funktion zu simulieren. Heute schauen wir uns Wachstum erstmal nicht aus der Perspektive der Differentialgleichungen an, sondern anhand der echten Werte die im Labor bestimmt werden. Außerdem erweitern wir das oben beschriebene Modell um einen weiteren Parameter, der ein unbegrenztes Wachstum verhindert. Denn wie Sie sich denken können ist Wachstum keineswegs ausschließlich exponentiell. 



<img src="https://raw.githubusercontent.com/CSBiology/BIO-MBP-06-U-2/ef274eba3125381a0da1f15a456077234da7bfdf/images/day2/Wachstumskurve.png" width=50% />


_Abbildung 1: Wachstumskurve. Verhalten der Zellzahl über die Zeit in einem einfachen Wachstumsexperiment._



_Aufgabe 1: Nennen Sie 2 Gründe warum eine lag-Phase existiert_
  


Antwort Aufgabe 1:

Schauen wir uns nun nochmals die Populationsgröße über die Zeit an, die gestern bestimmt wurde. Die Startzellzahl _N0_ ist 100 und die Wachstumskonstante _r_ ist 0.3. 

In [ ]:
/// Startzellzahl
let n0 = 100.
/// Wachstumskonstante
let r = 0.3

/// Funktion zur Berechnung von Zellzahl, abhängig zur Wachstumskonstante an einem bestimmten Zeitpunkt (t).
let getYValue t =
    n0*exp(r*t)

// Baue eine Liste von Zeitpunkten und der jeweiligen Zellzahl an dem Zeitpunkt von t = 0 bis t = 20 in ganzzahligen Schritten
[|
    for timepoint in [0. .. 20.] do
        timepoint,getYValue timepoint
|]
|> Chart.Spline // nutze die Liste als Datenpunkte für einen Spline Chart
|> Chart.withTitle("Abbildung 2 - Exponentielles Wachstum") // Chart Titel
|> Chart.withXAxisStyle("Time (t)") // Chart X-Axen Beschriftung
|> Chart.withYAxisStyle("Population size (N)") // Chart Y-Axen Beschriftung

<!-- Plotly chart will be drawn inside this DIV -->

_Aufgabe 2: Können Sie anhand des Graphen abschätzen wo lag und log-Phase liegen?_


Antwort Aufgabe 2:

### Datentransformation

Wie Sie vielleicht gemerkt haben, wächst die Population in obigem Beispiel zu jedem Zeitpunkt mit der gleichen Rate exponentiell. Wie besprochen hängt die Steigungsänderung nie von *t*, sondern nur von der gerade vorhandenen Zellzahl *N* und der Konstante *r* ab. In der Abbildung wirkt es jedoch, als würden sich die Zellen mit fortschreitender Zeit immer schneller teilen. Um diesen Effekt zu vermeiden sollten Zellzahlen grundsätzlich vor einer Wachstumsanalyse logarithmiert werden. Diese Transformation führt dazu, dass exponentielle Verhaltensweisen visuell korrigiert werden.

Ein Beispiel:

Ein Bakterium teilt sich jede Stunde genau ein mal. Nach der ersten Stunde sind es 2, nach der zweiten 4, nach der dritten 8 usw. Dieses Verhalten lässt sich durch $f(t)=2^t$ wie im Folgenden gezeigt darstellen.

_Aufgabe 3:_
  - Formulieren Sie die Funktion f(t), die dieses Verhalten widerspiegelt. Exponenten $x^y$ können in F# durch x**y dargestellt werden.
  - Beschriften Sie die Achsen des Diagramms


In [ ]:
/// Hier sollen Sie die Funktion f(t) einfügen. Sie wird verwendet zur Berechnung der Zellzahl an einem Zeitpunkt (t).
let myExponentialFunction t = 
    2.0**t //Aufabe! Bearbeite diese Zeile.

/// Baut eine Liste von Datenpunkten bestehend aus dem Zeitpunkt und der dazugehörigen Zellzahl.
let xyWerte = 
    [|
        // Benutzt werden die Zeitpunkte zwischen 0 und 24 in ganzzahligen Schritten: 0; 1; 2; 3 .. 24
        for t in [|0..24|] do  
            /// Der Graphen X-Wert ist der Zeitpunkt (t)
            let xWert = t
            /// Der Graphen Y-Wert ist das Ergebnis der Funktion "myExponentialFunction" angewendet auf einen Zeitpunkt (t). Die Zellzahl.
            let yWert = myExponentialFunction t
            xWert,yWert
    |]

Chart.Line xyWerte
//Aufabe! Bearbeite diese Zeile.

|> Chart.withTitle("Abbildung 3 - exponential growth") // Chart Titel

<!-- Plotly chart will be drawn inside this DIV -->

Allein durch das Betrachten einer exponentielle Funktion wird nicht klar, was zwischen Stunde 0 und Stunde 15 passiert. Um dafür einen Eindruck zu erhalten werden die y-Werte logarithmiert und erneut visualisiert.

In [ ]:
/// Hier erstellen wir eine Funktion die eine noch unbekannte Transformation auf den Y-Werten der "xyWerte" von dem vorherigen Codeblock anwendet.
/// Die Zeitpunkte (t) bleiben dabei unberührt.
/// Diese Funktion definieren wir um sie später nicht vollständig neu abtippen zu müssen für jede Transformation die wir durchführen wollen.
let werteTransformation (transformation: float -> float) = 
    [|
        for (x,y) in xyWerte do 
            let xWert = x
            let yWertTrans = transformation y
            xWert,yWertTrans
    |]

/// Durch Nutzung der "werteTransformation" wenden wir log2 auf den Y-Werten an.
let log2Transform = werteTransformation log2
/// Durch Nutzung der "werteTransformation" wenden wir log10 auf den Y-Werten an.
let log10Transform = werteTransformation log10
/// Durch Nutzung der "werteTransformation" wenden wir ln auf den Y-Werten an.
let lnTransform = werteTransformation log

// Hier erstellen wir eine List von Charts, basierend auf den transformierten Datenpunkten von zuvor.
// Zusätzlich geben wir bei der Erstellung der Graphen den Datenpunkten noch einen Namen.
[
Chart.Line (log2Transform,Name="log2")
Chart.Line (log10Transform,Name="log10")
Chart.Line (lnTransform,Name="ln")
]
|> Chart.combine // Wir kombinieren die Liste der Charts in einen einzelnen Charts. Alle Datenpunkte werden nun in einem einzigen Chart angezeigt.
|> Chart.withXAxisStyle("Time (h)")
|> Chart.withYAxisStyle("logX(Bakterienzahl)")
|> Chart.withTitle("Abbildung 4 - Logarithmus II")


<!-- Plotly chart will be drawn inside this DIV -->

Eine log-transformation (das logarithmieren), verringert hohe Werte deutlich stärker als kleine Werte. Die Interpretation des Verlaufs ist nun an jedem beliebigen x-Wert möglich, ohne dass man in den Plot "reinzoomen" muss. Aus exponentiellem Verhalten wird durch log-Transformation eine Gerade. Es wird deutlich, dass egal welche log Transformation angewandt wurde (log zur Basis 2, 10 oder der eulerischen Zahl (natürlicher Logarithmus)) Kurven zu Linien mit unterschiedlichen Steigung werden. 



_Aufgabe 4:_

  _a) Wie ist der untransformierte Wert bei Stunde 10 für alle drei transformationen? Hovern Sie über die Linien um den Funktionswert zu erhalten und rücktransformieren Sie den Logarithmus über_ $2^x, e^x oder 10^x$

  _b) Wie groß muss der y-Achsenabschnitt sein, um eine Verdopplung der Bakterien zu erhalten? Visuelle Abschätzung möglich._ 

  _c) Welchen Logarithmus würden Sie nutzen um Zellverdopplungen zu analysieren, und dadurch die visuelle Analyse so einfach wie möglich zu machen und warum?_ 



a)

  - Antwort log2:   

  - Antwort log10:  

  - Antwort ln:     

b)

  - Antwort log2:   

  - Antwort log10:  

  - Antwort ln:  

c)

  - Antwort:

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen:


### Expertenaufgaben
- Welcher Logarithmus wird bei pH-Werten genutzt und warum könnte dies als sinnvoll erachtet werden um zum Beispiel pH 2 direkt mit pH 5 vergleichen zu können?
- Wie groß ist der y-Achsenabschnitt im log space (log2 und log10) um von y=100 zu y=10000 zu kommen? Lösen Sie die Aufgabe erst durch schätzen, dann rechnerisch.
- Integrieren Sie $\frac{dN}{dt}=rN$ zu $N(t) = N_0 *e^{rt}$

___


## Biologisches Beispiel

Von einer Bakterienkultur wurde in regelmäßigen Abständen die Zellzahl bestimmt. Die gemessenen Werte können Sie aus untenstehender Tabelle entnehmen


|Zeitpunkt $(h)$|Zellzahl $(\frac{10^6}{ml})$|
|--|--|
|0.0|17|
|0.5|16.5|
|1.0|11|
|1.5|14|
|2.0|27|
|2.5|40|
|3.0|120|
|3.5|300|
|4.0|450|
|4.5|1,200|
|5.0|2,700|
|5.5|5,000|
|6.0|8,000|
|6.5|11,700|
|7.0|12,000|
|7.5|13,000|
|8.0|12,800|

In der folgenden Zelle wird die Zellzahl als Funktion der Zeit visualisiert.

Aufgabe 5:
  - Beschriften Sie in dem untenstehendem Diagramm die Achsen und Kurven.

In [ ]:
/// Eine Sequenz von Zeitpunkten von 0. bis 8. in 0.5 Schritten.
let time = [|0. .. 0.5 .. 8.|]

/// Eine Sammlung von gemessenen Zellzahlen.
let cellCount =
    [|
    17000000.;16500000.;11000000.;14000000.;27000000.;40000000.;120000000.;
    300000000.;450000000.;1200000000.;2700000000.;5000000000.;
    8000000000.;11700000000.;12000000000.;13000000000.;12800000000.
    |]

/// Für jeden Wert der gemessenen Zellzahlen "cellCount" wird der Logarithmus zur Basis 2 berechnet.
let cellCountLog2 = 
    [|
        for y in cellCount do log2 y
    |]

/// Erstellt einen Point Chart mit den Zeitpunkten "time" auf der X-Achse und den Werten von "cellCount" auf der Y-Achse.
/// Hierbei bleibt die Reihenfolge beider Sequenzen erhalten. 
let chartOrig = 
    Chart.Point(time,cellCount) 
    |> Chart.withTraceInfo "original data"  //Aufabe! Bearbeite diese Zeile.
    |> Chart.withYAxisStyle "cells"         //Aufabe! Bearbeite diese Zeile.

/// Erstellt einen Point Chart mit den Zeitpunkten "time" auf der X-Achse und den Werten von "cellCountLog2" auf der Y-Achse.
/// Hierbei bleibt die Reihenfolge beider Sequenzen erhalten. 
let chartLog =
    Chart.Point(time,cellCountLog2)
    |> Chart.withTraceInfo "log count"      //Aufabe! Bearbeite diese Zeile.
    |> Chart.withXAxisStyle "time (h)"      //Aufabe! Bearbeite diese Zeile. 
    |> Chart.withYAxisStyle "log2(cells)"   //Aufabe! Bearbeite diese Zeile.
    
let growthChart = 
    [chartOrig;chartLog] // Eine Liste mit den beiden oben erstellen Charts
    |> Chart.Grid(2,1) // Ordnet alle Charts einer Liste in ein Grid Schema ein. Hier: (Anzahl Reihen, Anzahl Spalten)
    |> Chart.withTitle("Abbildung 4 - Biologisches Beispiel")

// Der growthChart wird hier ausgeführt, damit er unterhalb des codeblocks dargestellt wird.
growthChart

<!-- Plotly chart will be drawn inside this DIV -->

Es ist ein deutlicher visueller Unterschied im Funktionsverlauf zu erkennen. Während im nicht-transformierten Diagramm der Zeitpunkt um 6 Stunden noch stark ansteigend (exponentielles Wachstum) erscheint, wird erst im transformierten Diagramm klar, dass die exponentielle Phase von Zeitpunkt 2 - 5 andauert und sich bei Stunde 6 schon am Übergang zum stationären Wachstum befindet. Durch die Abhängigkeit von Steigung und Zellzahl werden die echten Wachstumsphasen in untransformierten Daten kaschiert. Daher sollten Wachstumsdaten in der Regel im log-space angezeigt und analysiert werden. 

Wichtig: Im Folgenden wird immer davon ausgegangen, dass die Daten log transformiert wurden bevor die Kurven modelliert werden!

<img src="https://raw.githubusercontent.com/CSBiology/BIO-MBP-06-U-2/ef274eba3125381a0da1f15a456077234da7bfdf/images/day2/02_Growth_02.png" width=50% />


**Abbildung 6: Schema einer klassischen Wachstumskurve**

Wie im Schema gezeigt ist es sinnvoll die Daten zu fitten um bestimmte Parameter wie y0, ymax,  $\mu max$ und $\lambda$ ermitteln zu können. Ein einfaches Modell für exponentielles Wachstum haben wir bereits im ersten Abschnitt kennen gelernt. Würden wir auf die transformierten Daten (Abbildung 5 unten) nun jedoch das einfache Wachstumsmodell anwenden, dann würde es nicht zu einem sigmoidalen Verlauf kommen, sondern die Kurve würde immer weiter exponentiell ansteigen. Es gibt einige [Wachstumsmodelle](http://www.pisces-conservation.com/growthhelp/index.html), die diese Limitationen nicht aufweisen. Gesucht wird eine Erweiterung des einfachen Modells $\frac{dN}{dt} = rN$, die das Wachstum sowohl in der lag-, als auch in der stationären Phase begrenzen kann, also ein Faktor, der abhängig von der maximalen Zellzahl die Wachstumsgeschwindigkeit (growth rate) beschränkt:

$$\frac{dN}{dt}=N'=rN(1-\frac{N}{K})$$


wobei $K$ die so genannte **carrying capacity**, also die Maximalanzahl von Zellen, darstellt. Dieses Modell ist das Verhulst-Wachstumsmodell und beschreibt eine logistische Funktion mit unterer und oberer Asymptote.






_Aufgabe 6: Nennen Sie mindestens 3 Gründe für eine carrying capacity, bzw. das Vorhandensein einer stationären Phase._


Antwort Aufgabe 6: 

_Aufgabe 7: Erstellen Sie ein Modell der angepassten Differenzialgleichung, einem Wachstumsparameter "r" von 0.3, einer Maximalanzahl "k" von 20 und einer Startzellzahl von 1._

In [ ]:
let modelContext = 
    OdeContext(
        OdeSolverMethod.RK546M,
        OdeSolverOptions(
            StepSize=0.42
        )
    )
    
// Parameter
/// Wachstumsparameter
let r = 0.3     //Aufabe! Bearbeite diese Zeile.
/// Maximalanzahl
let K = 20.     //Aufabe! Bearbeite diese Zeile.

// Wachstumsmodell
let dN_dt_adj : SimpleModel =     
    fun N t ->
        /// Zellzahl
        let cellcount = N
        /// Zellwachstum
        let cellcount' = N * r * (1. - N / K) //Aufabe! Bearbeite diese Zeile.
        cellcount'

// Startbedingungen
/// Startzeitpunkt
let t0 = 0      //Aufabe! Bearbeite diese Zeile.
/// Startzellzahl
let N0 = 1.     //Aufabe! Bearbeite diese Zeile.

In [ ]:
// Simuliere das Modell mit den oben definierten Bedingungen.
let sim_dy_dx = 
    modelContext.OdeInt(t0,N0,dN_dt_adj)
    |> SolPoints.take 15 // Nimm die ersten 15 Zeitpunkte des Modells (oben als stepsize definiert).
    |> SolPoints.memorize

// Umwandlung der Ergebnisse in Datenpunkte die Zeitpuntkt und Zellzahl repräsentieren.
let verhulstCarryingCapacity =
    sim_dy_dx
    |> SolPoints.toPoints 1


verhulstCarryingCapacity
|> Chart.Point
|> Chart.withXAxisStyle("Time [h]")
|> Chart.withYAxisStyle("Zellzahl")
|> Chart.withSize(600.,400) // Definiert die Größe des Charts auf eurem Bildschirm.
|> Chart.withTitle("Abbildung 7 - Simulation verhulst growth model")


<!-- Plotly chart will be drawn inside this DIV -->

Mit diesem Modell können wir die stationäre Phase bereits gut abbilden. Eine komplexere Variation dieses Modells enthält auch noch einen Term für die lag-Phase. Letzteres soll später an die gegebenen logarithmierten Daten aus dem biologischen Beispiel gefittet werden. Ein mögliches sigmoidales Modell lautet:

$$log(N)=lmin + \frac{lmax - lmin}{1+e^{(k-t)/d}}$$

mit

$log(N)$ = Modellierung der logarithmierten Zellzahl

$lmin$ = untere asymptote

$lmax$ = obere asymptote/Zellmaximum

$k$ = x-Wert des Wendepunktes

$d$ = Steilheit

Diese Funktion heißt auch logistisches Modell. Zur Veranschaulichung wird die Funktion mit 4 verschiedene Parametersets visualisiert.

In [ ]:
/// Hier definieren wir die oben gezeigte Formel der Logistischen Funktion.
let logisticFunction lmin lmax (k:float) d t=
    lmin + (lmax - lmin)/(1.+ exp((k-t)/d))

/// Hier definieren wir eine Funktion die mit variablen Parametern für "lmin", "lmax", "k" und "d" die Logistische Funktion für die Zeitpunkte 0 bis 40 berechnet
/// und daraus einen Line Chart erstellt.
let plotLogistic lmin lmax k d =
    /// Baut eine Liste von Datenpunkten bestehend aus dem Zeitpunkt und der dazugehörigen Zellzahl.
    let data =
        [|
            // Benutzt werden die Zeitpunkte zwischen 0 und 40 in ganzzahligen Schritten: 0; 1; 2; 3 .. 40
            for x in [|0..40|] do
                /// Die Graphen X-Werte sind die Zeitpunkte (t)
                let timepoint = x
                /// Der Graphen Y-Wert ist das Ergebnis der Funktion "logisticFunction" angewendet auf einen Zeitpunkt (t). Die Zellzahl.
                let yValue = logisticFunction lmin lmax k d timepoint
                timepoint,yValue
        |]

    /// Erstellt Chart aus Datenpunkten "data" und benennt die Datenreihe abhängig von lmin, lmax, k und d
    let chart = 
        Chart.Line(data, Name=sprintf "min:%.0f max:%.0f k:%.0f d:%.0f" lmin lmax k d)
    chart

// Erstellt eine Liste von Line Charts verschiedener Logistischer Funktionen unter Anwendung der oben definierten Funktion "plotLogistic".
// Die zusätzlich gegeben Parameter entsprechen denen in der obigen Definition.
[
plotLogistic 0. 10. 4. 1.
plotLogistic 0. 10. 20. 1.
plotLogistic 0. 10. 20. 3.
plotLogistic 5. 20. 10. 3.
//plotLogistic x. y. u. v. //Aufabe! Bearbeite diese Zeile.
]
|> Chart.combine
|> Chart.withLegendStyle(X=0.6,Y=0.01) // Definiert die Position der Legende als X und Y Werte eines 2D-Koordinatensystems innerhalb des Plots.
|> Chart.withTitle("Abbildung 8 - Verschiedene logistische Funktionen")
|> Chart.withXAxisStyle("Time [h]")
|> Chart.withYAxisStyle("log(Zellzahl)")

<!-- Plotly chart will be drawn inside this DIV -->

_Aufgabe 8: Fügen sie eine weitere Modellierung mit folgenden Parametern hinzu. Ändern Sie hierfür einfach die auskommentierte Zeile im vorherigen Codeblock:_
  - _Minimum = 20_
  - _Maximum = 102_
  - _Zeitwert des Wendepunktes = 20_
  - _Steilheit = 5_

## Modelfitting

Nachdem der Funktionsverlauf nun bekannt ist versuchen wir dieses logistische Modell an die oben beschriebenen Daten (siehe biologisches Beispiel) zu fitten. Es müssen 4 Parameter bestimmt werden. Die untere Asymptote, die obere Asymptote, der Zeitwert des Wendepunktes und die Steilheit.

_Aufgabe 9: Schätzen Sie die vier Parameter und definieren Sie sie in dem nächsten Codeblock. Dadurch wird ein Diagramm mit den Rohwerten und dem logistischen Modell angezeigt das mit ihren Parametern beschrieben wurde. Versuchen Sie möglichst nah an die echten Daten zu kommen._
 

In [ ]:
// Parameter
/// lmax
let upperAsymptote =            23.  //Aufabe! Bearbeite diese Zeile.
/// lmin
let lowerAsymptote =            34.  //Aufabe! Bearbeite diese Zeile.
/// k
let xValueOfInflectionPoint =   3.9   //Aufabe! Bearbeite diese Zeile.
/// d
let steepness =                 1.  //Aufabe! Bearbeite diese Zeile.

/// Baut eine Liste von Datenpunkten bestehend aus dem Zeitpunkt und der dazugehörigen Zellzahl.
let plotMyData lmin lmax k d =
    let data=
        [|
            // Benutzt werden die Zeitpunkte zwischen 0. und 8. in 0.1 Schritten: 0.0; 0.1; 0.2; 0.3 .. 8.0
            for x in [|0. .. 0.1 .. 8.|] do
                /// Die Graphen X-Werte sind die Zeitpunkte (t)
                let timepoint = x
                /// Der Graphen Y-Wert ist das Ergebnis der Funktion "logisticFunction" angewendet auf einen Zeitpunkt (t). Die Zellzahl.
                let yValue = logisticFunction lmin lmax k d timepoint
                timepoint,yValue
        |]
    data 
    |> Chart.Line

[
Chart.Point(time,cellCountLog2) |> Chart.withTraceInfo "log2(N)" // Erstellt einen Point Chart der logarithmierten Rohdaten
// Erstellt den Fit der logistischen Funktion als Line Chart mit den oben definierten Parametern.
plotMyData upperAsymptote lowerAsymptote xValueOfInflectionPoint steepness |> Chart.withTraceInfo "manueller fit" 
]
|> Chart.combine
|> Chart.withTitle("Experimentell bestimmte Zellzahlen")

<!-- Plotly chart will be drawn inside this DIV -->

Diese manuelle Suche nach optimalen Parametern ist sehr aufwendig und natürlich im Laboralltag auch nicht notwendig. Hier nutzt man sogenannte Solver, die dieses Ausprobieren für einen erledigt.
Ein sogenannter Solver benötigt die Daten und das Modell, welches auf diese Daten appliziert werden soll. Der Solver mit dem Namen Levenberg-Marquardt versucht die Parameter des Modells so lange zu optimieren, bis der Abstand zwischen dem Modell und den echten Daten minimal geworden ist. Anschließend kann man die optimierten Parameter abfragen und mit diesen weiter arbeiten. Mehr Details finden Sie in *Vorlesung 13 - Principles of model fitting*. 

Beim Ausführen der nächsten Codezelle werden die Parameter automatisch optimal geschätzt und unter die Codezelle geschrieben.

In [ ]:
open System
open FSharp.Stats
open FSharp.Stats.Fitting.NonLinearRegression
open FSharp.Stats.Fitting.LinearRegression

/// Definiere eine Funktion zur Errechnung des optimalen Fits mit der oben definierten logistischen Funktion
let getLogisticParams timePoints logdata min max k d = 
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.verhulst4Param
        (createSolverOption 0.001 0.001 10000 [|max;k;d;min|])
        0.1
        10.
        timePoints
        logdata

let logisticParams =
    //getLogisticParams time cellCountLog2 23. 35. 3.5 1.
    getLogisticParams time cellCountLog2 23.391 33.598 4. 1.

// Zeigt die Ergebnisse des solvers an.
printfn "Durch cleveres Ausprobieren identifiziert der Solver folgende Parameter:
lmin = %.3f 
lmax = %.3f
k = %.3f
d = %.3f" logisticParams.[3] logisticParams.[0] logisticParams.[1] logisticParams.[2]



Durch cleveres Ausprobieren identifiziert der Solver folgende Parameter:
lmin = 23.578 
lmax = 33.987
k = 3.997
d = 0.956


Der folgende Codeblock dient nur der Visualiserung der Daten mit gefittetem Modell.

In [ ]:
let fittedValuesLogistic =
    /// Die Parameter wurden lokal bestimmt um Zeit während des Rechenprozesses zu sparen
    let f = Table.GrowthModels.verhulst4Param.GetFunctionValue logisticParams
    [|
        for x in [|time.[0] .. 0.1 .. Seq.last time|] do
            let timepoint = x
            let yValue = f timepoint
            timepoint,yValue
    |]
    |> Chart.Line

let fittedChartLogistic = 
    fittedValuesLogistic
    |> Chart.withXAxisStyle "time (h)" 
    |> Chart.withYAxisStyle "log2(cells/ml)"
    |> Chart.withTraceInfo "logistic"

let fittedChartLogisticS = 
    [
        Chart.Point(time,cellCountLog2)
        |> Chart.withTraceInfo "log2 count"
        fittedChartLogistic
    ]
    |> Chart.combine
    |> Chart.withXAxisStyle "time (h)" 
    |> Chart.withYAxisStyle "log2(cells/ml)"

fittedChartLogisticS

<!-- Plotly chart will be drawn inside this DIV -->

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen:

Aufgabe 10 
- Teilen Sie die Zeitpunkte in lag, log und stationäre Phase ein.

### Expertenaufgaben
- Wie sind im gezeigten Fall die Werte der unteren und oberen Asymptote? Geben Sie die Werte als untransformierte Zellzahl an.

___

## Verdopplungszeit-Bestimmung

Wie bereits herausgefunden, ist eine Zellverdopplung im log2 space gleichzusetzen mit einem y-Achsenabschnitt von 1. Um nun die Generations/Verdopplungs-Zeit zu bestimmen muss die Zeit bestimmt werden in dem sich die log(Zellzahl) um 1 erhöht. Dies kann durch die maximale Steigung ($\mu max$) des gefitteten Modells erreicht werden. Diese entspricht dem schnellstmöglichen Wachstum und ist naturgemäß am Wendepunkt (inflection point) der sigmoidalen Kurve. Diese Steigung wird auch als "maximal growth rate" oder "specific growth rate" bezeichnet und wird gerne genutzt um Unterschiede zwischen Experimenten zu quantifizieren <sup>[1](https://doi.org/10.1016/j.fm.2004.11.014)</sup>. 

_Aufgabe 11: Schätzen Sie ab, wo sich der Wendepunkt - also der Punkt maximaler Steigung - befindet._

Antwort:

Da im logistischen Modell ein Parameter schon optimiert wurde, der genau den x-Wert des Wendepunktes definiert, muss man nur noch diesen Wert (k = 3.997) in die Ableitung einsetzen um die maximale Steigung zu erhalten

$$f(t)=lmin + \frac{lmax - lmin}{1+e^{(k-t)/d}}$$


$$f'(t)=\frac{(lmax-lmin)e^{(k-t)/d}}{d(e^{(k-t)/d}+1)^2}$$

optimierte Parameter aus Model-Fitting:

$$lmin = 23.578$$

$$lmax = 33.987$$

$$k = 3.997$$

$$d = 0.956$$

Die Parameter werden im Folgenden eingesetzt um den Funktionswert und die Steigung am Wendepunkt zu erhalten. 

$$f(3.997) = 23.578 + \frac{33.987 - 23.578}{1+e^{(3.997-3.997)/0.956}} = 28.782$$

$$f'(3.997) = \frac{(33.987-23.578)e^{(3.997-3.997)/0.956}}{0.956(e^{(3.997-3.997)/0.956}+1)^2} = \frac{33.987 - 23.578}{0.956*2^2} = 2.72$$

Vergleiche diese Werte visuell mit der letzten Abbildung. Die maximale Steigung wurde berechnet und ist 2.72. 

Um nun die Verdopplungszeit zu bestimmen muss der Logarithmus von 2. einfach durch diese Steigung geteilt werden. Da der Logarithmus von 2 zur Basis 2 genau 1 ergibt, ist es im Fall einer log2-Transformation besonders einfach. Hier ergibt einfach der Kehrwert der maximalen Steigung die Generationszeit. 




<img src="https://raw.githubusercontent.com/CSBiology/BIO-MBP-06-U-2/ef274eba3125381a0da1f15a456077234da7bfdf/images/day2/GenTime.png" width=50% />


In [ ]:
let generationtime  =
    log2(2.) / 2.72
 
printfn "The generation time (logistic model) is: %.1f min" (generationtime * 60.)


The generation time (logistic model) is: 22.1 min


Die bestimmte Verdopplungszeit von 22 Minuten kann nun genutzt werden um sie gegen Verdopplungszeiten anderer Experimente zu vergleichen.

### Anleitung Generationszeitbestimmung
1. Plotten Sie die log2 transformierten Daten als xy Diagramm.
2. Wenn ein Wachstumsmodell und Solver vorhanden ist, fitten Sie das Modell an die Daten und isolieren sie $\mu max$.
3. Wenn kein Wachstumsmodell vorhanden ist, schätzen Sie den linearen Bereich der Wachstumskurve.
    - Erstellen Sie einen Ausgleichsgerade von diesem Bereich und isolieren Sie die Steigung $\mu max$.
4. Um die Generationszeit zu bestimmen rechnen Sie $\frac{log_x 2.}{\mu max} = \frac{1}{\mu max}$ wobei x die Basis angibt zu der die Originalwerte log-transformiert wurden.

<img src="https://raw.githubusercontent.com/CSBiology/BIO-MBP-06-U-2/main/images/day2/Tutorial.png" width=50% />


### Take home messages
- Die logistische Wachstumsfunktion $N(t)=lmin + \frac{lmax - lmin}{1+e^{(k-t)/d}}$ hat niemand einfach entwickelt, sondern es ist ein Integral eines Wachstumsmodells.
- Wenn Wachstum untersucht werden soll, sollte man sich den zeitlichen Verlauf (fast) immer mit log-transformierten Zellkonzentrationen anschauen.
- Bei log2 transformierten Daten ist ein y-Achsenabschnitt von 1 gleichzusetzen mit einer Verdopplung der Zellzahl.
- Die Verdopplungszeit hängt ausschließlich von der Steigung der log-phase ($\mu max$ oder _maximal growth rate_) ab. Sie wird berechnet mit $\frac{log_x 2}{\mu max}$ wobei x die gewählte Basis ist, mit der die Zellzahlen transformiert wurden.
- Die Steigung kann entweder durch manuelle Selektion der log phase und anschließendes Fitten einer Gerade geschehen, oder durch fitten eines Wachstumsmodells ([Gompertz/Richards/Verhulst](https://fslab.org/FSharp.Stats/GrowthCurve.html)).



### References
- Maier R., Pepper I., Chapter 3 - Bacterial Growth, Environmental Microbiology (Third Edition), 2015, https://doi.org/10.1016/B978-0-12-394626-3.00003-X
- Perni S., Andrew P., Shama G., Estimating the maximum growth rate from microbial growth curves: definition is everything, Food Microbiology, 2005, https://doi.org/10.1016/j.fm.2004.11.014
- Venn B., Schneider K., Weil L., Zimmer D. and Mühlhaus T., fslaborg/FSharp.Stats: Release 0.4.7, 2022, Growth curve documentation: https://fslab.org/FSharp.Stats/GrowthCurve.html
- Growth II, version 2.3.6.71, PISCES Conservation Ltd United Kingdom



### Expertenaufgaben
- Wie ist die Verdopplungszeit? Die Zeitpunkte sind als Tage angeben.
  - a) $\mu max = 2.5$; Basis der log-Transformation: 2
  - b) $\mu max = 1.0$; Basis der log-Transformation: e
  - c) $\mu max = 0.7$; Basis der log-Transformation: 10
  - d) $\mu max = 10$; Basis der log-Transformation: e
  - e) $\mu max = 10$; Basis der log-Transformation: 2
  - f) $\mu max = 10$; Basis der log-Transformation: 10


